In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_hub as hub
import cv2

In [2]:
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

In [3]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1)

In [4]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [5]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)

In [6]:
# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

In [10]:
# Load the Video File
cap = cv2.VideoCapture('video.mp4') 

In [ ]:
 # If to do for webcam just remove the video file and put 0 which is default for webcam.
while cap.isOpened():
    ret, frame = cap.read()
    
    # Define the polygon points coordinates
    # polygon_points = np.array([[917, 251], [367, 250], [88, 717], [1165, 718]]) 
    # polygon_points = np.array([[1000, 400], [274, 398], [88, 717], [1165, 718]])
    # polygon_points = np.array([[1095, 340], [190, 337], [85, 719], [1165, 718]])

    # Create a binary mask
    # mask = np.zeros((int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)), int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), 3), dtype=np.uint8)
    # cv2.fillPoly(mask, [polygon_points], (1, 1, 1))
    
    # Apply the mask
    # frame = cv2.multiply(frame, mask)
    
    # Resize image
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 352,640)
    input_img = tf.cast(img, dtype=tf.int32)
   
    # Detection section
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    
    # Render keypoints 
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.1)
    
    cv2.imshow('Movenet Multipose', frame)
    
    # Detect keypoints
    detector = cv2.ORB_create()
    keypoints = detector.detect(frame, None)

    # Get pixel coordinates of keypoints
    for kp in keypoints:
        x, y = kp.pt
    print("Keypoint at ({}, {})".format(x, y))
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

Keypoint at (687.970947265625, 326.0695495605469)
Keypoint at (716.6363525390625, 286.654541015625)
Keypoint at (816.9654541015625, 182.74227905273438)
Keypoint at (687.970947265625, 326.0695495605469)
Keypoint at (788.2999877929688, 182.74227905273438)
Keypoint at (648.555908203125, 268.7386474609375)
Keypoint at (1114.3695068359375, 200.65818786621094)
Keypoint at (788.2999877929688, 182.74227905273438)
Keypoint at (472.9800109863281, 515.9782104492188)
Keypoint at (691.5540771484375, 182.74227905273438)
Keypoint at (458.64727783203125, 451.4809265136719)
Keypoint at (548.226806640625, 182.74227905273438)
Keypoint at (691.5540771484375, 182.74227905273438)
Keypoint at (476.5632019042969, 519.5613403320312)
Keypoint at (1114.3695068359375, 200.65818786621094)
Keypoint at (770.3840942382812, 591.2250366210938)
Keypoint at (1114.3695068359375, 200.65818786621094)
Keypoint at (691.5540771484375, 182.74227905273438)
Keypoint at (716.6363525390625, 286.654541015625)
Keypoint at (1114.36950

Keypoint at (1039.1226806640625, 121.82818603515625)
Keypoint at (519.5613403320312, 250.82272338867188)
Keypoint at (1114.3695068359375, 214.99090576171875)
Keypoint at (741.7186279296875, 605.5577392578125)
Keypoint at (748.885009765625, 584.0586547851562)
Keypoint at (666.4718017578125, 283.0713806152344)
Keypoint at (386.983642578125, 494.4790954589844)
Keypoint at (666.4718017578125, 283.0713806152344)
Keypoint at (687.970947265625, 419.2322692871094)
Keypoint at (537.477294921875, 329.6527404785156)
Keypoint at (505.2286376953125, 250.82272338867188)
Keypoint at (745.3018188476562, 286.654541015625)
Keypoint at (666.4718017578125, 283.0713806152344)
Keypoint at (490.8959045410156, 286.654541015625)
Keypoint at (691.5540771484375, 182.74227905273438)
Keypoint at (369.0677185058594, 505.2286376953125)
Keypoint at (716.6363525390625, 569.7258911132812)
Keypoint at (515.9782104492188, 286.654541015625)
Keypoint at (154.07681274414062, 372.6509094238281)
Keypoint at (781.1336669921875

Keypoint at (781.1336669921875, 182.74227905273438)
Keypoint at (781.1336669921875, 182.74227905273438)
Keypoint at (605.5577392578125, 340.40228271484375)
Keypoint at (813.3822631835938, 182.74227905273438)
Keypoint at (154.07681274414062, 369.0677185058594)
Keypoint at (515.9782104492188, 386.983642578125)
Keypoint at (691.5540771484375, 182.74227905273438)
Keypoint at (154.07681274414062, 351.1518249511719)
Keypoint at (261.572265625, 150.49363708496094)
Keypoint at (598.391357421875, 286.654541015625)
Keypoint at (788.2999877929688, 182.74227905273438)
Keypoint at (781.1336669921875, 182.74227905273438)
Keypoint at (781.1336669921875, 182.74227905273438)
Keypoint at (781.1336669921875, 182.74227905273438)
Keypoint at (548.226806640625, 182.74227905273438)
Keypoint at (788.2999877929688, 182.74227905273438)
Keypoint at (788.2999877929688, 182.74227905273438)
Keypoint at (644.9727172851562, 254.40591430664062)
Keypoint at (558.9763793945312, 318.9031982421875)
Keypoint at (508.811828

Keypoint at (1053.4554443359375, 157.66000366210938)
Keypoint at (433.56500244140625, 250.82272338867188)
Keypoint at (1003.2908935546875, 261.572265625)
Keypoint at (265.15545654296875, 182.74227905273438)
Keypoint at (548.226806640625, 182.74227905273438)
Keypoint at (265.15545654296875, 182.74227905273438)
Keypoint at (548.226806640625, 182.74227905273438)
Keypoint at (641.3895263671875, 290.23773193359375)
Keypoint at (623.4736328125, 544.6436157226562)
Keypoint at (390.56683349609375, 250.82272338867188)
Keypoint at (644.9727172851562, 551.8099975585938)
Keypoint at (419.2322692871094, 250.82272338867188)
Keypoint at (637.806396484375, 591.2250366210938)
Keypoint at (265.15545654296875, 182.74227905273438)
Keypoint at (616.3072509765625, 555.3931884765625)
Keypoint at (143.3272705078125, 218.5740966796875)
Keypoint at (143.3272705078125, 218.5740966796875)
Keypoint at (548.226806640625, 182.74227905273438)
Keypoint at (379.8172912597656, 261.572265625)
Keypoint at (343.98547363281

Keypoint at (150.49363708496094, 164.8263702392578)
Keypoint at (591.2250366210938, 182.74227905273438)
Keypoint at (591.2250366210938, 182.74227905273438)
Keypoint at (1053.4554443359375, 118.24500274658203)
Keypoint at (781.1336669921875, 182.74227905273438)
Keypoint at (781.1336669921875, 182.74227905273438)
Keypoint at (566.1427612304688, 279.4881896972656)
Keypoint at (1128.7022705078125, 268.7386474609375)
Keypoint at (781.1336669921875, 182.74227905273438)
Keypoint at (408.48272705078125, 250.82272338867188)
Keypoint at (139.7440948486328, 121.82818603515625)
Keypoint at (781.1336669921875, 182.74227905273438)
Keypoint at (612.72412109375, 268.7386474609375)
Keypoint at (569.7258911132812, 254.40591430664062)
Keypoint at (401.3163757324219, 286.654541015625)
Keypoint at (139.7440948486328, 121.82818603515625)
Keypoint at (1143.0350341796875, 268.7386474609375)
Keypoint at (139.7440948486328, 121.82818603515625)
Keypoint at (616.3072509765625, 369.0677185058594)
Keypoint at (404.

Keypoint at (429.9818115234375, 340.40228271484375)
Keypoint at (429.9818115234375, 340.40228271484375)
Keypoint at (863.5468139648438, 254.40591430664062)
Keypoint at (150.49363708496094, 161.24317932128906)
Keypoint at (616.3072509765625, 268.7386474609375)
Keypoint at (1103.6199951171875, 168.4095458984375)
Keypoint at (1049.872314453125, 157.66000366210938)
Keypoint at (1164.5340576171875, 211.40773010253906)
Keypoint at (852.7973022460938, 293.8209228515625)
Keypoint at (852.7973022460938, 293.8209228515625)
Keypoint at (616.3072509765625, 268.7386474609375)
Keypoint at (616.3072509765625, 268.7386474609375)
Keypoint at (429.9818115234375, 340.40228271484375)
Keypoint at (429.9818115234375, 340.40228271484375)
Keypoint at (616.3072509765625, 268.7386474609375)
Keypoint at (616.3072509765625, 268.7386474609375)
Keypoint at (429.9818115234375, 340.40228271484375)
Keypoint at (150.49363708496094, 164.8263702392578)
Keypoint at (584.0586547851562, 361.9013671875)
Keypoint at (623.4736

Keypoint at (268.7386474609375, 182.74227905273438)
Keypoint at (386.983642578125, 326.0695495605469)
Keypoint at (268.7386474609375, 182.74227905273438)
Keypoint at (1082.1209716796875, 164.8263702392578)
Keypoint at (548.226806640625, 182.74227905273438)
Keypoint at (548.226806640625, 182.74227905273438)
Keypoint at (548.226806640625, 182.74227905273438)
Keypoint at (139.7440948486328, 121.82818603515625)
Keypoint at (781.1336669921875, 182.74227905273438)
Keypoint at (849.214111328125, 283.0713806152344)
Keypoint at (953.1264038085938, 268.7386474609375)
Keypoint at (845.6309204101562, 304.5704650878906)
Keypoint at (996.1245727539062, 465.8136291503906)
Keypoint at (548.226806640625, 182.74227905273438)
Keypoint at (956.7095336914062, 283.0713806152344)
Keypoint at (247.2395477294922, 175.57591247558594)
Keypoint at (781.1336669921875, 182.74227905273438)
Keypoint at (687.970947265625, 333.23590087890625)
Keypoint at (963.8759155273438, 340.40228271484375)
Keypoint at (960.29272460

Keypoint at (318.9031982421875, 315.32000732421875)
Keypoint at (813.3822631835938, 182.74227905273438)
Keypoint at (566.1427612304688, 444.3145446777344)
Keypoint at (566.1427612304688, 444.3145446777344)
Keypoint at (186.32545471191406, 136.16090393066406)
Keypoint at (1114.3695068359375, 164.8263702392578)
Keypoint at (877.8795776367188, 182.74227905273438)
Keypoint at (480.1463623046875, 268.7386474609375)
Keypoint at (533.8941040039062, 322.4863586425781)
Keypoint at (591.2250366210938, 182.74227905273438)
Keypoint at (326.0695495605469, 308.15362548828125)
Keypoint at (612.72412109375, 369.0677185058594)
Keypoint at (591.2250366210938, 390.56683349609375)
Keypoint at (154.07681274414062, 372.6509094238281)
Keypoint at (691.5540771484375, 182.74227905273438)
Keypoint at (154.07681274414062, 372.6509094238281)
Keypoint at (154.07681274414062, 372.6509094238281)
Keypoint at (154.07681274414062, 372.6509094238281)
Keypoint at (591.2250366210938, 182.74227905273438)
Keypoint at (548.2

Keypoint at (691.5540771484375, 254.40591430664062)
Keypoint at (548.226806640625, 182.74227905273438)
Keypoint at (781.1336669921875, 182.74227905273438)
Keypoint at (781.1336669921875, 182.74227905273438)
Keypoint at (781.1336669921875, 182.74227905273438)
Keypoint at (691.5540771484375, 290.23773193359375)
Keypoint at (756.0513916015625, 404.8995361328125)
Keypoint at (781.1336669921875, 182.74227905273438)
Keypoint at (265.15545654296875, 333.23590087890625)
Keypoint at (594.8081665039062, 250.82272338867188)
Keypoint at (1006.8740844726562, 114.66181945800781)
Keypoint at (910.128173828125, 211.40773010253906)
Keypoint at (1003.2908935546875, 111.0786361694336)
Keypoint at (745.3018188476562, 286.654541015625)
Keypoint at (139.7440948486328, 121.82818603515625)
Keypoint at (816.9654541015625, 182.74227905273438)
Keypoint at (422.8154602050781, 293.8209228515625)
Keypoint at (154.07681274414062, 372.6509094238281)
Keypoint at (816.9654541015625, 182.74227905273438)
Keypoint at (139

Keypoint at (781.1336669921875, 182.74227905273438)
Keypoint at (809.799072265625, 286.654541015625)
Keypoint at (283.0713806152344, 311.73681640625)
Keypoint at (637.806396484375, 182.74227905273438)
Keypoint at (781.1336669921875, 182.74227905273438)
Keypoint at (548.226806640625, 182.74227905273438)
Keypoint at (899.378662109375, 283.0713806152344)
Keypoint at (899.378662109375, 283.0713806152344)
Keypoint at (139.7440948486328, 121.82818603515625)
Keypoint at (548.226806640625, 182.74227905273438)
Keypoint at (1006.8740844726562, 114.66181945800781)
Keypoint at (723.802734375, 437.148193359375)
Keypoint at (139.7440948486328, 121.82818603515625)
Keypoint at (587.641845703125, 182.74227905273438)
Keypoint at (773.96728515625, 437.148193359375)
Keypoint at (587.641845703125, 182.74227905273438)
Keypoint at (494.4790954589844, 315.32000732421875)
Keypoint at (322.4863586425781, 340.40228271484375)
Keypoint at (548.226806640625, 182.74227905273438)
Keypoint at (218.5740966796875, 390.5